<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Simple Decision Prompts

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundemental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal of prompt engineering is to "tune" the prompt in a way that maximizes the quality and clarity of the AI's response, often using specific wording, context, or examples within the prompt itself.

Decision Intelligence and Prompt Engineering techniques can be combined to created "Generative AI Decision Intellignce"! This can leverage GenAI models to reason over complex tasks, gather intelligence for decisions, recommend decisions, communicate decisions etc. 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.54"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.54.0

Using Azure OpenAI Service


### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [3]:
// A Decision Intelligence prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
var simplePromptResponseString = simplePromptResponse.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
simplePromptResponseString.DisplayAs("text/markdown");

Here are five decision-making frameworks that can enhance the quality of decisions, along with a brief description of how they support better analysis and reasoning in various scenarios:

---

### 1. **The Pugh Matrix (Weighted Scoring Model)**
   - **Description**: The Pugh Matrix is a decision-making tool used to compare and evaluate multiple options against a set of weighted criteria. Each option is scored on how well it satisfies the criteria, and the scores are tallied to identify the most viable choice.
   - **How It Enhances Analysis**: By assigning weights to criteria and evaluating each option objectively, the Pugh Matrix helps eliminate bias and ensures decisions are based on measurable factors. It’s especially useful when trade-offs are involved, such as choosing between competing business strategies or product designs.

---

### 2. **The Cynefin Framework**
   - **Description**: The Cynefin Framework categorizes decision-making contexts into five domains: simple, complicated, complex, chaotic, and disorder. Each domain suggests different approaches to problem-solving and decision-making.
   - **How It Enhances Analysis**: This framework helps decision-makers identify the nature of a given situation, ensuring the use of context-appropriate strategies. For instance, structured processes might work in simple situations, while experimentation and adaptability are necessary in complex or chaotic scenarios.

---

### 3. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description**: SWOT Analysis involves assessing internal strengths and weaknesses alongside external opportunities and threats to inform strategic decisions.
   - **How It Enhances Analysis**: This framework provides a structured way to evaluate a situation comprehensively, making it easier to align decisions with organizational capabilities and external challenges. It’s particularly effective in business planning and prioritizing initiatives.

---

### 4. **The OODA Loop (Observe, Orient, Decide, Act)**
   - **Description**: Developed by military strategist John Boyd, the OODA Loop emphasizes a continuous cycle of observing data, orienting to the situation, making decisions, and executing actions.
   - **How It Enhances Analysis**: The cyclical nature of the OODA Loop encourages dynamic decision-making, allowing individuals or teams to adapt quickly to changing conditions. It’s highly effective in fast-paced environments, such as crisis management or competitive markets.

---

### 5. **Six Thinking Hats**
   - **Description**: Developed by Edward de Bono, this method involves adopting six metaphorical "hats" representing different perspectives (e.g., logical, emotional, creative, cautious, optimistic, and neutral) for decision-making.
   - **How It Enhances Analysis**: By intentionally analyzing a problem from varied perspectives, this framework encourages balanced reasoning and prevents overreliance on any single mode of thinking. It’s particularly useful for brainstorming, group decisions, and innovative problem-solving.

---

Each of these frameworks provides tools for structuring analysis, fostering objectivity, and systematically addressing the complexities inherent in decision-making, leading to higher-quality outcomes across various scenarios.

### Step 3 - Execute a Decision Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> 📝 Note: An average human can read between 25-40 Tokens / second. Therefore, wile streaming certainly helps with providing AI output to the user, it begins to lose its effectivness at large token velocity.  

In [4]:
// Decision Prompt using Streaming output chunks 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

Here are five widely recognized decision-making frameworks that can help improve decision quality by providing structured approaches to analysis and reasoning:

### 1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description**: SWOT Analysis is a straightforward framework used to evaluate the internal strengths and weaknesses of a decision or project, as well as the external opportunities and threats it faces.
   - **How It Supports Better Analysis**: By breaking down internal and external factors, SWOT facilitates a balanced view of the situation, enabling decision-makers to leverage strengths, mitigate weaknesses, capitalize on opportunities, and anticipate potential threats. It encourages considering both positive and negative aspects comprehensively.

---

### 2. **Pros and Cons Analysis**
   - **Description**: This framework involves listing the advantages (pros) and disadvantages (cons) of each option under consideration.
   - **How It Supports Better 

### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can ouput Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting ouptut. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [ ]:
// A new decision prompt to help with describing decision-making frameworks using Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Do not enclose the table in triple backticks.
""";

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptWithMarkdownFormat))
{
   Console.Write(streamChunk);
}

| Decision-Making Framework   | Description                                                                                       |
|-----------------------------|---------------------------------------------------------------------------------------------------|
| SWOT Analysis              | Helps identify Strengths, Weaknesses, Opportunities, and Threats, providing a structured way to assess internal and external factors. |
| Decision Matrix             | Offers a systematic method by assigning weights and scores to criteria, helping prioritize options objectively. |
| Cost-Benefit Analysis       | Compares the costs and benefits of a decision quantitatively, aiding in evaluating its feasibility and potential return.         |
| Six Thinking Hats           | Encourages examining a decision from multiple perspectives (e.g., facts, emotions, risks), promoting balanced and creative analysis. |
| 80/20 Rule (Pareto Analysis)| Focuses on identifying the most impactful 20% of factors that contribute to 80% of outcomes, enhancing efficiency in decision-making.|

### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Tempature or other settings.

> 📝 Note: The supported settings are dependendent on the API plus the specific model version. For example, a model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available.  

In [7]:
// Create a new OpenAI Prompt Execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// KernelArguments to pass the OpenAI Prompt Execution settings
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var streamOutputvsMarkdownFormat = false; // True will stream and false will wait and display the response as Markdown
if (streamOutputvsMarkdownFormat)
{
    await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
    {
    Console.Write(streamChunk);
    }
}
else
{
    var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
    var simplePromptResponseString = simplePromptResponse.GetValue<string>();

    // Display the response from the Semantic Kernel as Markdown
    simplePromptResponseString.DisplayAs("text/markdown");
}

Here are five decision-making frameworks that are widely recognized for enhancing the quality of decisions:

---

### 1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description**: SWOT is a structured decision-making framework that evaluates internal and external factors affecting a situation. It helps analyze strengths and weaknesses (internal factors) and opportunities and threats (external factors).
   - **How It Supports Analysis**: SWOT promotes a balanced view by encouraging consideration of both positive and negative aspects of a decision or strategy. It helps identify competitive advantages, areas for improvement, future opportunities, and potential risks, offering a comprehensive overview of a scenario.

---

### 2. **Cost-Benefit Analysis (CBA)**
   - **Description**: CBA involves comparing the costs and benefits of a decision to determine whether the benefits outweigh the costs.
   - **How It Supports Analysis**: By quantifying costs and benefits (monetary or qualitative), CBA ensures decisions are rooted in data and logic rather than intuition alone. It simplifies complex decisions by focusing on overall net value, enabling better resource allocation and prioritization.

---

### 3. **The Eisenhower Matrix (Urgent-Important Matrix)**
   - **Description**: This framework helps prioritize tasks by categorizing them into four quadrants: Urgent-Important, Not Urgent-Important, Urgent-Not Important, and Not Urgent-Not Important.
   - **How It Supports Analysis**: The matrix helps decision-makers focus on what matters most (important but not urgent) while minimizing distractions from less critical tasks. It enhances time management and productivity, especially for prioritizing actions in busy or dynamic environments.

---

### 4. **Decision Tree Analysis**
   - **Description**: A decision tree is a visual tool that maps possible choices, outcomes, probabilities, and payoffs in a decision-making process.
   - **How It Supports Analysis**: By breaking down complex problems into smaller, manageable parts, decision trees provide clarity on each option and its associated risks. They also help evaluate outcomes under uncertainty and support evidence-based reasoning through a logical structure.

---

### 5. **Six Thinking Hats**
   - **Description**: This framework, developed by Edward de Bono, requires participants to view decisions through six distinct perspectives or "hats" (white for facts, red for emotions, black for risks, yellow for optimism, green for creativity, and blue for process management).
   - **How It Supports Analysis**: The method encourages a holistic approach by considering logical, emotional, and creative factors systematically. It prevents groupthink, fosters diverse viewpoints, and ensures well-rounded analysis by addressing emotional, creative, and analytical dimensions of decision-making.

---

These frameworks promote structured thinking, reduce biases, and provide clarity for making sound decisions across various scenarios, from strategic planning to problem-solving and resource prioritization.

### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decison Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific (i.e. decision intelligence)
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [8]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

Console.WriteLine("PROMPT TO SEND TO THE CHAT COMPLETION SERVICE...");
Console.WriteLine(string.Empty);
Console.WriteLine(simpleDecisionPromptTemplate);
Console.WriteLine(string.Empty);
Console.WriteLine("----------------------------------------");
Console.WriteLine(string.Empty);
Console.WriteLine("RESPONSE FROM THE CHAT COMPLETION SERVICE...");
Console.WriteLine(string.Empty);

// Execute the Decision Intelligence prompt with the Semantic Kernel
var streamOutputvsMarkdownFormat = false; // True will stream and false will wait and display the response as Markdown
if (streamOutputvsMarkdownFormat)
{
    await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
    {
    Console.Write(streamChunk);
    }
}
else
{
    var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
    var simplePromptResponseString = simplePromptResponse.GetValue<string>();

    // Display the response from the Semantic Kernel as Markdown
    simplePromptResponseString.DisplayAs("text/markdown");
}

PROMPT TO SEND TO THE CHAT COMPLETION SERVICE...

System Prompt: 
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.

Request from the user: 
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.

----------------------------------------

RESPONSE FROM THE CHAT COMPLETION SERVICE...



Here are five decision-making frameworks that are simple, versatile, and can be applied to a wide range of daily situations:

---

### 1. **The Eisenhower Matrix (Urgent-Important Matrix)**
   - **Best for**: Managing time and prioritizing tasks or decisions.
   - **How it works**:
     - Draw a 2x2 grid with "Urgent" vs. "Not Urgent" on one axis and "Important" vs. "Not Important" on the other.
     - Categorize tasks or decisions into one of the four quadrants:
       1. **Urgent & Important (Do now)** – Handle immediately.
       2. **Not Urgent & Important (Schedule)** – Plan for later.
       3. **Urgent & Not Important (Delegate)** – Delegate or minimize effort.
       4. **Not Urgent & Not Important (Eliminate)** – Ignore or delete.
   - **Why it works**: It forces you to focus on what truly matters and avoid wasting time on trivial activities.

---

### 2. **The OODA Loop (Observe, Orient, Decide, Act)**
   - **Best for**: Quick decision-making in dynamic or uncertain situations.
   - **How it works**:
     - **Observe**: Gather relevant information about the situation.
     - **Orient**: Analyze the data and understand your options.
     - **Decide**: Choose the best course of action based on your goals.
     - **Act**: Execute your decision, then loop back to observe the outcome.
   - **Why it works**: It encourages continuous feedback and adaptation, which helps improve decisions over time.

---

### 3. **The 10/10/10 Rule**
   - **Best for**: Evaluating the short-, medium-, and long-term impacts of a decision.
   - **How it works**:
     - Ask yourself:
       1. How will I feel about this decision in **10 minutes**?
       2. How will I feel about it in **10 months**?
       3. How will I feel about it in **10 years**?
     - Use these perspectives to assess whether the decision aligns with your long-term values and goals.
   - **Why it works**: It prevents impulsive decisions by expanding your thought process beyond the immediate context.

---

### 4. **The 80/20 Rule (Pareto Principle)**
   - **Best for**: Identifying high-impact decisions or actions.
   - **How it works**:
     - Recognize that **80% of results often come from 20% of causes**.
     - Focus your energy and decision-making on the small percentage of actions (key 20%) that yield the most significant results (vital 80%).
     - For example, prioritize the few tasks that drive the most impact in your work or personal life.
   - **Why it works**: It helps you focus on what truly matters, maximizing efficiency and effectiveness.

---

### 5. **The "WOOP" Method (Wish, Outcome, Obstacle, Plan)**
   - **Best for**: Turning decisions into actionable plans.
   - **How it works**:
     1. **Wish**: Define your objective or desired result (what do you want?).
     2. **Outcome**: Visualize the best result if your wish is fulfilled.
     3. **Obstacle**: Identify what could stand in your way (internal or external).
     4. **Plan**: Create a "if-then" strategy to overcome obstacles and achieve your goal.
     - Example: "If I get distracted, then I'll set a 10-minute timer to refocus."
   - **Why it works**: It makes decision-making actionable and prepares you for potential setbacks.

---

### Bonus: **The Two-Minute Rule**
   - **Best for**: Deciding whether to act on small tasks or defer them.
   - **How it works**: If an action will take **less than two minutes**, do it immediately rather than overthinking or procrastinating.
   - **Why it works**: Eliminates clutter and helps you stay on top of small but important daily decisions.

---

These frameworks are not mutually exclusive—you can combine them depending on the complexity and context of your decisions. They help minimize overwhelm, prioritize effectively, and make confident choices in your daily life.

### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full Decision Intelligence framework will not be used, rather a simple request for Artificial Intelligene to recommend a path forward (recommedation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [9]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed Decision Scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommedation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, // Increase the max tokens to allow for a more detailed response
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var streamOutputvsMarkdownFormat = false; // True will stream and false will wait and display the response as Markdown
if (streamOutputvsMarkdownFormat)
{
    await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(scenarioDecisionPromptTemplate, kernelArguments))
    {
    Console.Write(streamChunk);
    }
}
else
{
    var simplePromptResponse = await semanticKernel.InvokePromptAsync(scenarioDecisionPromptTemplate);
    var simplePromptResponseString = simplePromptResponse.GetValue<string>();

    // Display the response from the Semantic Kernel as Markdown
    simplePromptResponseString.DisplayAs("text/markdown");
}

## Recommendation for Alex: Direct Enrollment in the Four-Year University

| **Decision Factors**                | **Reasoning**                                                                                                  | **Impact**                                                                                         |
|-------------------------------------|--------------------------------------------------------------------------------------------------------------- |----------------------------------------------------------------------------------------------------|
| **Financial Considerations**        | Although the four-year university is more expensive, the family can afford this with some loans. Starting at a community college could save substantial money, but the medium concern about cost does not outweigh the benefits of avoiding transfer complexities. | Over time, a four-year university degree potentially offers a higher return on investment (ROI), given networking and career opportunities available earlier.|
| **Career and Major Uncertainty**    | Starting at a four-year university could help Alex narrow down her major through access to advisors, career-focused programs, and structured academic pathways designed for exploration. While choosing the wrong major might lead to extra time and cost, many universities allow flexibility in this regard. Community college’s exploratory nature might seem appealing but adjusting to a new system midway through could add stress. | Direct enrollment ensures consistency in academic advising for major exploration and avoids potential disruptions tied to transferring institutions.|
| **Academic Consistency and Networking** | A four-year university would allow Alex to establish early relationships with professors, peers, and professional contacts through internships or research opportunities. This consistency could be beneficial in shaping her professional trajectory. Community college might delay her ability to access rich educational resources and long-term mentorship. | Early networking and closer proximity to campus resources improve Alex’s chances of building a strong foundation for career opportunities earlier in her academic journey.|
| **Future Impact**                  | A four-year university provides a more seamless path to Alex's goals of graduating with a complete network and relevant experience. Starting at community college may affect her ability to build these relationships and could limit her ability to take full advantage of four-year university offerings due to a delayed start. While student loans might be an issue, manageable debt for a degree from a well-regarded university is often mitigated by better career prospects after graduation. | Early exposure to a large, professionally focused network and substantial institutional resources improves her long-term career viability and helps minimize transitional hurdles.|

### **Overall Recommendation:**
Alex should enroll directly in the **four-year university**. Despite higher upfront costs, this pathway offers her a seamless, consistent academic journey, key opportunities to build professional networks, early access to resources, and a structured environment to explore majors. The long-term career benefits outweigh the medium financial concern. By planning her academic trajectory thoughtfully within the four-year university, she can mitigate risks associated with uncertain career choices while ensuring she graduates on time with valuable experiences and connections.